In [83]:
from transformers import AutoTokenizer, AutoModelForCausalLM

polyglot_repo_id = "EleutherAI/polyglot-ko-1.3b"
repo_id = "rycont/KoQuestionBART"
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer_2 = AutoTokenizer.from_pretrained(polyglot_repo_id)
model = AutoModelForCausalLM.from_pretrained(repo_id)
model_2 = AutoModelForCausalLM.from_pretrained(polyglot_repo_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForCausalLM were not initialized from the model checkpoint at rycont/KoQuestionBART and are newly initialized: ['lm_head.weight', 'model.decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]


In [84]:
sentence = "(서울=연합뉴스) 김잔디 오진송 권지현 기자 = '빅5'로 불리는 서울시내 주요 대형병원 중 서울대병원과 서울아산병원에 이어 나머지 병원도 주 1회 전면 휴진에 동참할 가능성이 커지고 있다."
# sentence = "Using a Transformer network is simple"
tokens = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
tokens_2 = tokenizer_2(sentence, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [86]:
tokens["input_ids"]

tensor([[24570, 14116, 12063, 10012, 14075, 12335, 11331, 14472, 12332, 13672,
         14680, 15146, 14063, 11029,   252, 16374, 18783, 16422,  9517, 14979,
         15582, 16892, 14059, 17780, 10877, 16435, 14245, 11696, 11211, 10877,
         15288, 14311, 17382, 17787,  9866, 14053, 19254, 17253, 14888, 21650,
         20237, 13594, 15367, 20696, 15964]])

In [87]:
tokens_2["input_ids"]

tensor([[   11,  2701,    32,  2571,  4883,    12,   657, 22587,   540,   473,
           960,  1255, 20260,  1720,  5297,   514,  6334,    24,    10,   286,
          3247,   272,  2674,   574,  1689,  2168,  2077,   469,  3966,  2077,
           359,   933, 12206,  2077,   274,  1350,  3290,  2326,   309,   458,
           376,   437,  3529,  1646,   473,   274,  5104,   482,   997, 12304,
          3940,   283,   327,   267,    17]])

In [152]:
model.name_or_path

'rycont/KoQuestionBART'

In [89]:
model_2

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
  

In [91]:
outputs = model(tokens["input_ids"])

In [95]:
len(outputs.past_key_values)

6

In [96]:
len(outputs.past_key_values[0])

2

In [97]:
len(outputs.past_key_values[0][0])

1

In [105]:
outputs.past_key_values[0][0].shape

torch.Size([1, 16, 45, 48])

In [109]:
tokens["input_ids"][0].shape

torch.Size([45])

In [110]:
outputs.logits.shape

torch.Size([1, 45, 30000])

In [111]:
import torch

In [122]:
indices = torch.argmax(outputs.logits, -1)

In [123]:
tokenizer.decode(indices[0])

'닺 이색닺 합리적 기사 김대逃이라도橐켸ol 슈팅벤呉浦浦=갓 자본주의松 화제가 열흘 자본주의자에게는툭 자본주의 자본주의 자본주의 자본주의 자본주의 자본주의 캠페인을 자본주의 김민툭연구툭松== 했는데松 자본주의 자본주의雛'

In [118]:
outputs_2 = model_2(tokens_2["input_ids"])

indices_2 = torch.argmax(outputs_2.logits, -1)

In [119]:
tokenizer_2.decode(indices_2[0])

'1=연합뉴스) 임계 기자수진희 기자숙 기자 = "최순뱅\' 병원 불리는 대형내 주요 사립서점들이에서병원과 삼성아산병원이 이어 세 병원들 신종초회 이상파업진에 돌입한다 것성이 높고 있다. 이'

In [141]:
param_count = 0
params = list(model_2.parameters())    

In [142]:
for p in params:
    temp_count = 1
    for elem in list(p.size()):
        temp_count *= elem

    param_count += temp_count

In [143]:
param_count

1331810304

In [146]:
p.numel()

61603840

In [147]:
p.size()

torch.Size([30080, 2048])